# **4. Building Models with PyTorch**

parameter를 제외한 모든 클래스는 torch.nn.Module의 하위 클래스임 이 클래스는 파이토치 모델과 그 구성 요소에 특화도니 동작을 캡슐화 하도록 설계된 파이토치 기본 클래스임

torch.nn.Module의 중요한 동작 중 하나는 파라미터 등록임

만약 어떤 모듈 클래스가 학습 가능한 가중치를 가진다면 이 가중치는 torch.nn.Parameter 인스턴스로 표현됨

In [ ]:
import torch

class TinyModel(torch.nn.Module):

  def __init__(self):
    super(TinyModel, self).__init__()

    self.linear1 = torch.nn.Linear(100, 200)
    self.activation = torch.nn.ReLU()
    self.linear2 = torch.nn.Linear(200, 10)
    self.softmax = torch.nn.Softmax()

  def forward(self, x):
    x = self.linear1(x)
    x = self.activation(x)
    x = self.linear2(x)
    x = self.softmax(x)
    return x

tinymodel = TinyModel()

print('Model:')
print(tinymodel)

print('\n\n Just One layer:')
print(tinymodel.linear2)

print('\n\n Model Params:')
for param in tinymodel.parameters():
  print(param)

print('\n\nLayer params:')
for param in tinymodel.linear2.parameters():
  print(param)

Model:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)


 Just One layer:
Linear(in_features=200, out_features=10, bias=True)


 Model Params:
Parameter containing:
tensor([[ 0.0253,  0.0231,  0.0601,  ..., -0.0821, -0.0310,  0.0845],
        [ 0.0021,  0.0274, -0.0558,  ...,  0.0193,  0.0664,  0.0204],
        [ 0.0284, -0.0784, -0.0757,  ...,  0.0590,  0.0711, -0.0973],
        ...,
        [-0.0264, -0.0486, -0.0788,  ..., -0.0181, -0.0014,  0.0489],
        [ 0.0738,  0.0836,  0.0392,  ..., -0.0027, -0.0815,  0.0357],
        [ 0.0312,  0.0549, -0.0012,  ...,  0.0999, -0.0534, -0.0494]],
       requires_grad=True)
Parameter containing:
tensor([-0.0739, -0.0076, -0.0209,  0.0937, -0.0022,  0.0037, -0.0568,  0.0324,
        -0.0621, -0.0646,  0.0314,  0.0666, -0.0882, -0.0183, -0.0636, -0.0690,
         0.0441,  0.0002, -0.0788,  0.0364

# Common Layer Types

신경망의 가장 기본적인 유형의 계층은 선형 계층 (Linear layer) 또는 Fully connected layer 임

이 계층은 모든 입력이 출력의 모든 요소에 영향을 미치며 가중치로 그 영향이 결정될거임



In [ ]:
# 선형 계층, 입력차원 3 / 출력차원 2
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3) # 입력 텐서
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters(): # 선형 계층의 파라미터 출력
  print(param)

# 선형 계층 통과
y = lin(x)
print('\n\nOutput:')
print(y) # 출력

Input:
tensor([[0.0604, 0.3293, 0.3864]])


Weight and Bias parameters:
Parameter containing:
tensor([[ 0.5087,  0.2750, -0.2794],
        [-0.5630,  0.5327, -0.0600]], requires_grad=True)
Parameter containing:
tensor([ 0.4664, -0.1808], requires_grad=True)


Output:
tensor([[ 0.4797, -0.0625]], grad_fn=<AddmmBackward0>)


# Convolution Layers

합성곱 계층은 이미지의 특징을 인식하기 위해 자주 사용

In [ ]:
import torch.functional as F

class LeNet(torch.nn.Module):

  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = torch.nn.Conv2d(1,6,5) # 입력 채널 1, 출력 채널 6, 커널 크기 5 x
    self.conv2 = torch.nn.Conv2d(6, 16, 3) # 입력 채널 6, 출력 채널 16, 커널 크기 3 x 3
    self.fc1 = torch.nn.Linear(16 * 6 * 6, 120) # 입력 576, 출력 120
    self.fc2 = torch.nn.Linear(120, 84)
    self.fc3 = torch.nn.Linear(84, 10) # 10개 클래스

  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), 2)
    x = x.view(-1, self.num_flat_features(x))
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc(x))
    x = self.fc3(x)
    return x

  # Flatten 시에 필요한 feature 수 계산
  def num_flat_features(self, x):
    size = x.size()[1:] # 배치 차원 제외
    num_features = 1
    for s in size:
      num_features *= s
    return num_features

# Recurrnet Layers

RNN은 시퀀스 데이터에 적합

RNN은 시퀀스를 따라 기억을 유지하면서 다음 단계로 전달함


In [ ]:
class LSTMTagger(torch.nn.Module):

  def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
    super(LSTMTagger, self).__init__()
    self.hidden_dim = hidden_dim

    # 단어 임베딩 정의 (어휘 수, 임베딩 차원)
    self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

    # LSTM 계층 : 임베딩을 입력으로 받아 hidden_dim 크기의 상태 출력
    self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

    # 출력층 : hidden state -> tag 분류
    self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

  def forward(self, sentence):
    # 임베딩 처리 : 각 단어를 임베딩 벡터로 매핑
    embeds = self.word_embeddings(sentence)

    # LSTM에 입력 -> (시퀀스 길이, 배치, 임베딩 차원)
    lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))

    # hidden2tag : (시퀀스 길이, 태그 수)
    tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

    # Log_softmax로 확률 출
    tag_scores = F.log_softmax(tag_space, dim=1)
    return tag_scores

# Data Manipulation Layers

딥러닝 모델에는 학습 가능 파라미터는 없지만 중요한 역할을 수행하는 계층들이 있음

pooling layer에서는 텐서의 여러 셀을 하나로 합쳐 축소시키는 역할을 함



In [ ]:
my_tensor = torch.rand(1, 6 ,6) # 무작위 텐서 생성
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3) # 3 x 3 필터로 maxpooling 수행
print(maxpool_layer(my_tensor)) # 2 x 2로 축소된 것 확인가능

tensor([[[0.0648, 0.6371, 0.9348, 0.8121, 0.1974, 0.6994],
         [0.9895, 0.4340, 0.6323, 0.4568, 0.3168, 0.5022],
         [0.3403, 0.6135, 0.0097, 0.0473, 0.1302, 0.0443],
         [0.6599, 0.7203, 0.6938, 0.7566, 0.8212, 0.1226],
         [0.8400, 0.3933, 0.3167, 0.6692, 0.4682, 0.9360],
         [0.3327, 0.9790, 0.0202, 0.8790, 0.8615, 0.4834]]])
tensor([[[0.9895, 0.8121],
         [0.9790, 0.9360]]])


정규화는 다음 계층에 전달하기 전에 출력을 정규화해서 학습을 안정화 시킴

활성화 함수의 기울기가 0 근처에서 가장 크므로, 입력 분포를 0 주변으로 모으면 학습 속도와 안정성이 향상된다


In [ ]:
# 평균 5, 표준편차 20 정도로 스케일된 텐서
my_tensor = torch.rand(1, 4, 4) * 20 + 5
print(my_tensor)
print(my_tensor.mean()) # 평균이 13임

norm_layer = torch.nn.BatchNorm1d(4) # 채널 개수 4개로 배치 정규화
normed_tensor = norm_layer(my_tensor)
print(normed_tensor)
print(normed_tensor.mean()) # 평균이 0에 가까워짐

tensor([[[ 9.7749, 12.3121,  7.5704,  5.7735],
         [ 9.7683, 17.8252, 14.6655,  7.8687],
         [20.3475, 11.1342, 22.9355,  9.1526],
         [22.3060, 14.5677, 14.7394, 21.0780]]])
tensor(13.8637)
tensor([[[ 0.3749,  1.4119, -0.5262, -1.2606],
         [-0.7022,  1.3450,  0.5421, -1.1849],
         [ 0.7598, -0.8115,  1.2012, -1.1495],
         [ 1.1655, -1.0165, -0.9681,  0.8192]]],
       grad_fn=<NativeBatchNormBackward0>)
tensor(4.8429e-08, grad_fn=<MeanBackward0>)


드랍아웃은 학습 시에 입력 텐서의 일부 요소를 확률적으로 0으로 만들어 과적합을 줄이고 희소 표현을 유도. 인퍼런스 시에는 자동으로 비활성화


In [ ]:
my_tensor = torch.rand(1, 4, 4)

dropout = torch.nn.Dropout(p=0.4) # 40% 확률로 드랍아웃
print(dropout(my_tensor)) # 일부 값들이 0으로 마스킹 된것을 확인가능
print(dropout(my_tensor)) # 호출할 때마다 마스크가 바뀜

tensor([[[1.5612, 0.3873, 1.3936, 0.4430],
         [0.0000, 0.7739, 1.0800, 0.0000],
         [0.8307, 0.7257, 0.0000, 1.1206],
         [0.0000, 0.3563, 0.0000, 0.5511]]])
tensor([[[1.5612, 0.0000, 1.3936, 0.4430],
         [1.1498, 0.7739, 1.0800, 0.6723],
         [0.0000, 0.7257, 0.9893, 0.0000],
         [0.0000, 0.0000, 0.8240, 0.5511]]])


activation function, 즉 활성화 함수는 신경망에 선형 연산만 반복하게 되면 사실 네트워크 전체가 하나의 큰 행렬 곱으로 축약되어 버림

비선형 활성화 함수를 층 사이에 삽입해야 복잡한 비선형 함수를 근사할 수 있음


손실 함수는 모델 예측과 정답 사이의 오차를 수치화해 학습 방향을 제시함